In [7]:
from langchain_community.retrievers import BM25Retriever        # pip install rank_bm25
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
import keyring

doc_list = [
    '우리나라는 2022년에 코로나가 유행했다.',
    '우리나라의 2024년 GDP 전망은 3.0%이다.',
    '우리나라는 2022년 국내총생산 중 연구개발 예산은 약 5%이다.'
]

# pip install rank_bm25
# elastic search 단어 기반 검색 : embedding 필요 없음 
bm25_retriever = BM25Retriever.from_texts(
    doc_list, 
    metadatas=[{"source":1}] * len(doc_list)    # 메타데이터 소스 정보 임의 생성
)      
bm25_retriever.k = 1    # 가상 유사한 결과 1개 리턴

In [10]:
embedding = OpenAIEmbeddings(api_key=keyring.get_password('openai', 'key_for_windows'))
chroma_vectorstores = Chroma.from_texts(
    doc_list, embedding, metadatas=[{"source":1}] * len(doc_list)
)
chroma_retriever = chroma_vectorstores.as_retriever(search_kwargs={"k":1})

In [11]:
faiss_vectorstores = FAISS.from_texts(
    doc_list, embedding, metadatas=[{'source':1}] * len(doc_list) 
)
faiss_retriever = faiss_vectorstores.as_retriever(search_kwargs={'k':1})

In [12]:
query = "2022년 우리나라 GDP대비 R&D 규모는?"

bm25_docs = bm25_retriever.invoke(query)
chroma_docs = chroma_retriever.invoke(query)
faiss_docs = faiss_retriever.invoke(query)

print(bm25_docs)
print('--------------')
print(chroma_docs)
print('--------------')
print(faiss_docs)

[Document(metadata={'source': 1}, page_content='우리나라는 2022년 국내총생산 중 연구개발 예산은 약 5%이다.')]
--------------
[Document(metadata={'source': 1}, page_content='우리나라는 2022년 국내총생산 중 연구개발 예산은 약 5%이다.')]
--------------
[Document(metadata={'source': 1}, page_content='우리나라는 2022년 국내총생산 중 연구개발 예산은 약 5%이다.')]
